In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import heapq
import matplotlib.pyplot as plt
from IPython.display import Audio as player
import librosa
# !pip install playsound
# import playsound
# !pip install pyaudio
# import pyaudio


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
print('Pandas version: ', pd.__version__)

import numpy as np
print('NumPy version: ', np.__version__)

import matplotlib
print('Matplotlib version: ', matplotlib.__version__)

from matplotlib import pyplot as plt

import sklearn
print('Scikit-Learn version: ', sklearn.__version__)

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.cluster import KMeans


import pickle
print('Pickle version: ', pickle.format_version)

import sys
print('Sys version: ', sys.version[0:5])

from sys import exc_info

import ast

In [613]:
id_dataset = pd.read_csv("/kaggle/input/music-vectors/id_vectors-5.csv")

In [ ]:
meta_dataset = pd.read_csv("/kaggle/input/filtered-final-metadata-4/filtered-final-metadata-6.csv")
meta_dataset

In [ ]:
dummy_metaID = len(id_dataset)
dummy_metaID

In [ ]:
simR = np.load('/kaggle/input/similarity-matrices/raag-similarity-matrix-mean.npy')
simM = np.load('/kaggle/input/similarity-matrices/meta-similarity-matrix-1.npy')

zero_row = np.zeros((1, simM.shape[1]))  # One row with same number of columns as sim_meta
zero_col = np.zeros((simM.shape[0]+1, 1))  # One column with same number of rows as sim_meta

simM = np.concatenate([simM, zero_row], axis=0)  # Add row
simM = np.concatenate([simM, zero_col], axis=1)  # Add column



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming similarity_matrix is your 2D similarity matrix
# Generate a sample similarity matrix for demonstration


# Flatten the simiRlarity matrix into a 1D array and remove diagonal elements
similarities = simR.flatten()
similarities = similarities[similarities != 1.0]  # Remove self-similarities

# Plot the frequency distribution
plt.figure(figsize=(8, 6))
plt.hist(similarities, bins=20, edgecolor='black', alpha=0.7)
plt.xlabel('Similarity')
plt.ylabel('Frequency')
plt.title('Frequency Distribution of Similarities')
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming similarity_matrix is your 2D similarity matrix
# Generate a sample similarity matrix for demonstration


# Flatten the simiRlarity matrix into a 1D array and remove diagonal elements
similarities = simM.flatten()
similarities = similarities[similarities != 1.0]  # Remove self-similarities

# Plot the frequency distribution
plt.figure(figsize=(8, 6))
plt.hist(similarities, bins=20, edgecolor='black', alpha=0.7)
plt.xlabel('Similarity')
plt.ylabel('Frequency')
plt.title('Frequency Distribution of Similarities')
plt.grid(True)
plt.show()


In [ ]:
import pandas as pd

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('/kaggle/input/artist-map/artist-map-3.csv')
artist_dict = {}
# Convert the DataFrame to a dictionary
for _,row in df.iterrows():
    try:
        artist_id = row['artist_id']
        artist_name = row['artist_name']

        artist_dict[str(artist_id) ] = artist_name.strip()
    except:
        continue
len(artist_dict)
artist_dict

In [ ]:
import json
import numpy as np

# Load JSON data from file
file_path = '/kaggle/input/song-similarities/normalized_similarity_dict (1).json'
with open(file_path, 'r') as f:
    simA = json.load(f)



In [ ]:
import os

def get_audio_file_names(directory):
    """
    Get the names of audio files (without extensions) from the specified directory.

    Args:
    - directory (str): Path to the directory containing audio files.

    Returns:
    - file_names (list): List of audio file names (without extensions) in the directory.
    """
    file_names = []
    # Iterate over all files in the directory
    for file_name in os.listdir(directory):
        # Check if the file is an audio file
        if file_name.endswith('.mp3') or file_name.endswith('.wav'):
            # Remove the extension and add the file name to the list
            file_names.append(os.path.splitext(file_name)[0])
    return file_names

audio_files=get_audio_file_names('/kaggle/input/darbar-audio/darbar_audio')
len(audio_files)

In [ ]:
import json

def load_scores(path):
    # Load the JSON file
    with open('/kaggle/input/model-song-score/scores.json', 'r') as json_file:
        sim = json.load(json_file)
    return sim

# Load the scores
sim = load_scores("/kaggle/input/model-song-score/scores.json")


In [ ]:
best_fit={}
def compute_best_fits(sim, audio_files):

    global best_fit
    for i in range(len(audio_files)):
        max_obs = -float('inf')
        file_name = audio_files[i]
        for j in range(len(audio_files)):
            if i != j:
                other_file_name = audio_files[j]
                try:
                    score_diff = sim[f'(\'{file_name}\', \'{other_file_name}\')']
                    if score_diff > max_obs:
                        max_obs = score_diff                        
                except KeyError:
                    continue
        best_fit[file_name]=max_obs

In [ ]:
import math

coeff = 1/1000000
def getAudioSimilarity(songID1,songID2):
    try:
        rec1 = song_name.loc[int(songID1),'Name'] + 'mp'
        rec2 = song_name.loc[int(songID2),'Name'] + 'mp'
        global best_fit
        sim12 = (-2 / math.pi * math.atan(math.pi * coeff *abs(sim[f'(\'{rec1}\', \'{rec2}\')']) / 2) + 1)
        sim21 = (-2 / math.pi * math.atan(math.pi * coeff *abs(sim[f'(\'{rec2}\', \'{rec1}\')']) / 2) + 1)
        return (sim12+sim21)/2
    except:
        return 0

In [ ]:
maxsofar = -1
for i in range(313):
    maxsofar = max(maxsofar,getAudioSimilarity(5,i))
  
maxsofar

In [ ]:
rec_sim('55','55',['55','44','5'])

In [ ]:
def rec_sim(rec1,rec2,user):

    rec1_split = ""
    user_split = ""
    rec2_split = ""
    
    
    if rec1 != "":
        rec1_split = rec1.split('|')
    if user[2] != "":
        user_split = user[0].split('|')
    if rec2 != "":
        rec2_split = rec2.split('|')
    
    sim = 0
    
    length = (len(rec1_split)+len(user_split)) * len(rec2_split)
    if length == 0:
        return 0
    
    for recID in rec1_split:
        for recID2 in rec2_split:
            sim += getAudioSimilarity(recID,recID2)

    for recID in user_split:
        for recID2 in rec2_split:
            sim +=getAudioSimilarity(recID,recID2)/5
    return sim/length


In [ ]:
file_path = '/kaggle/input/filtered-final-metadata-4/filtered-final-metadata-6.csv'

# Read the CSV file into a DataFrame
song_name = pd.read_csv(file_path)

# Display the DataFrame
print(song_name)
size = len(song_name)

In [ ]:
artist_sim('26','25',['1','25','34'])

In [576]:
artist_dict['761']

'Ulhas Kashalkar'

In [ ]:
print(artist_sim('761','761',['','','']))
print(artist_sim('414','761',['','','']))


In [623]:
def artist_sim(artist1,artist2,user):

    artist1_split = ""
    user_split = ""
    artist2_split = ""
    
    
    if artist1 != "":
        artist1_split = artist1.split('|')
    if user[1] != "":
        user_split = user[1].split('|')
    if artist2 != "":
        artist2_split = artist2.split('|')
    
    sim = 0
    length1 = len(artist1_split)*len(artist2_split)
    length2 = (len(user_split))*len(artist2_split)
   
    try:
        for artistID in artist1_split:
            for artistID2 in artist2_split:
                artist1_name = artist_dict[artistID]
                artist2_name = artist_dict[artistID2]
                if artist1_name == artist2_name:
                    sim+=1
                else:
                    sim+= simA[f'(\'{artist1_name}\', \'{artist2_name}\')']/length1
    
    except:
        sim =0
    
    try:
        for artistID in user_split:
            for artistID2 in artist2_split:
                artist1_name = artist_dict[artistID]
                artist2_name = artist_dict[artistID2]
                if artist1_name == artist2_name:
                    sim+=1
                else:
                    sim+= simA[f'(\'{artist1_name}\', \'{artist2_name}\')'] / (5*length2)
    except:
        sim+=sim
    return sim/4


In [ ]:
raag_sim('1','23',['313','21',''])

In [ ]:
def raag_sim(raga1,raga2,user):
    raga1_split = ""
    user_split = ""
    raga2_split = ""
    
    if raga1 != "":
        raga1_split = raga1.split('|')
    if raga2 != "":
        raga2_split = raga2.split('|')
    if user[2] != "":
        user_split = user[2].split('|')
    
    sim = 0
    
    length1 = len(raga1_split)*len(raga2_split)
    length2 = (len(user_split))*len(raga2_split)
    
    for ragaID in raga1_split:
        for ragaID2 in raga2_split:
            sim += simR[int(ragaID),int(ragaID2)]/length1
    for ragaID in user_split:
        for ragaID2 in raga2_split:
            sim += simR[int(ragaID),int(ragaID2)] / (5*length2)
            
    return sim/2

In [ ]:
meta_sim('3','5',['313|5','2',''])

In [ ]:
def meta_sim(meta1,meta2,user):
    try:
        meta1_split = ""
        user_split = ""
        meta2_split = ""
    
        if meta1 != "":
            meta1_split = meta1.split('|')
        if user[0] != "":
            user_split = user[0].split('|')
        if meta2 != "":
            meta2_split = meta2.split('|')
        sim=0
        
        #length = (len(meta1_split)+len(user_split)-1)*len(meta2_split)
        length1 = len(meta1_split)*len(meta2_split)
        length2 = (len(user_split))*len(meta2_split)
        
    
        for metaID in meta1_split:
            for metaID2 in meta2_split:
                sim += simM[int(metaID),int(metaID2)]/length1
        for metaID in user_split:
            for metaID2 in meta2_split:
                sim += simM[int(metaID), int(metaID2)] /(5*length2)
            
        return sim/2
    except:
        return 0

In [ ]:
song_name.loc[int(3),'Name']

In [609]:
class User:
    
    def __init__(self, user_id):
        self.user_id = user_id
        #self.liked_recs = []
        self.liked_recs = ["","",""]
    
    def like_rec(self, rec):
        for i in range(len(self.liked_recs)):
            if self.liked_recs[i] == "":
                self.liked_recs[i]+=str(rec[i])
            else:
                self.liked_recs[i] +="|"+str(rec[i])        #if rec not in self.liked_recs:
         #   self.liked_recs.append(rec)


        
    def get_liked_recs(self):
        return self.liked_recs

    
    def user_rep(self,artistID,ragaID):
        user_rep = []
        user_rep.append(str(dummy_metaID))
        user_rep.append(str(artistID))
        user_rep.append(str(ragaID))
        
        self.like_rec(user_rep)

In [558]:
id_dataset

,meta_id,artist_id,raag_id
0,0,13,102
1,1,13,107
2,2,13,74
3,3,13,57
4,4,13,27
...,...,...,...
308,308,779,82
309,309,779,82
310,310,779,90
311,311,792,109


In [616]:
def giveRecsForNewVec(vec,user_rep):
    
        
    priority_queue = []
    
    for index,rec in id_dataset.iterrows():
            
        score =(rec_sim(str(vec[0]),str(rec['meta_id']),user_rep) + meta_sim(str(vec[0]),str(rec['meta_id']),user_rep)+artist_sim(str(vec[1]),str(rec['artist_id']),user_rep) +raag_sim(str(vec[2]),str(rec['raag_id']),user_rep))/4
        
        heapq.heappush(priority_queue, (-score,rec['meta_id']))  #max-heap
    
    #Display in descending order, using audio player allow the user to play the rec.
    recs = []

    for i in range(min(len(priority_queue),10)):
        recs.append(heapq.heappop(priority_queue))
    return recs

In [560]:
meta_dataset

,Raga,Duration,Artist,Instrument,Laya,Taal,Accompanists,Instruments,Form,Name
0,Sahana,474,Abhisek Lahiri,Sarod,Madhya,Jhaptaal,Sanju Sahai,Tabla,Khyal,RaagSahanaAbhisekLahiriMusicofIndia
1,Shree,713,Abhisek Lahiri,Sarod,Drut,Teentaal,Sukhvinder Singh 'Pinky',Tabla,Khyal,AbhisekLahiriSukhvinderSinghPinkyRaagShreeSaro...
2,Malkauns,247,Abhisek Lahiri,Sarod,Drut,NaN,NaN,NaN,Jor|Jhala,SpellbindingRaagMalkaunsAbhisekLahiriMusicofIndia
3,Kaunsi Kanada,59,Abhisek Lahiri,Sarod,Drut,NaN,NaN,NaN,Khyal,RaagKaunsiKanadaAbhisekLahiriMusicofIndia
4,Chayanat,598,Abhisek Lahiri,Sarod,Madhya,Roopak,Sukhvinder Singh 'Pinky',Tabla,Khyal,AbhisekLahiriSukhvinderSinghPinkyRaagChayanatS...
...,...,...,...,...,...,...,...,...,...,...
308,Multani,657,Venkatesh Kumar,Vocal,Vilambit,Ektaal,Murad Ali Khan|Satyajit Talwalkar|Tofail Ahmed,Sarangi|Tabla|Harmonium,Khyal,AmazingRaagMultaniPanditVenkateshKumarSarangiH...
309,Multani,335,Venkatesh Kumar,Vocal,Drut,Teentaal,Satyajit Talwalkar|Murad Ali|Tofail Ahmed,Tabla|Sarangi|Harmonium,Khyal,PowerfulKhayalRaagMultaniPanditVenkateshKumarM...
310,Patdeep,327,Venkatesh Kumar,Vocal,Madhya,Jhaptal,Murad Ali Khan|Satyajit Talwalkar|Tofail Ahmed,Sarangi|Tabla|Harmonium,Khyal,MajesticRaagPatdeepVenkateshKumarKhayalvocal
311,Shuddha Sarang,429,Vinayak Torvi,Vocal,Madhya,Teentaal,Kousic Sen|Bharat Bhushan Goswami|Ajay Joglekar,Tabla|Sarangi|Harmonium,Khyal,PanditVinayakTorviRaagShudhSarangKhayalVocal


In [575]:
rec_listraag = getRaagRec('Shree',['313','25','12'])
print(artist_dict['25'])
for i, rec in enumerate(rec_listraag):
    getAudioDetails(rec[1])


Ajoy Chakraborty
 Id:8, Artist : Ajoy Chakraborty , Main Instrument : Vocal , Raga : Shree , Taal : EktaalTeentaal , Laya : MadhyaVilambit
 Id:66, Artist : Deborshee Bhattacharya , Main Instrument : Vocal , Raga : Shree , Taal : EktaalTeentaal , Laya : Vilambit
 Id:107, Artist : Kaustuv Kanti Ganguli , Main Instrument : Vocal , Raga : Shree , Taal : EktaalJhaptaalTeentaal , Laya : MadhyaVilambit
 Id:138, Artist : Manjusha Kulkarni-Patil , Main Instrument : Vocal , Raga : Shree , Taal : nan , Laya : Vilambit
 Id:179, Artist : Omkar Dadarkar , Main Instrument : Vocal , Raga : Shree , Taal : TeentaalTilwada , Laya : MadhyaVilambit


In [620]:
def getRaagRec(raag,user_rep):
    
    # Filter rows where the value in the "Raga" column equals the given raag
    raag_rows = meta_dataset[meta_dataset['Raga'] == raag]
    
    # Get the indexes of the filtered rows
    indexes = raag_rows.index.tolist()
    
    priority_queue = []
    
    customdf = []
    
    for meta_id in indexes:
        row =id_dataset.loc[meta_id].tolist()
        customdf.append(row)
    
    #print(customdf)
    id_data = pd.DataFrame(customdf, columns=['meta_id','artist_id','raag_id'])
    for index,rec in id_data.iterrows():
            
        score =(rec_sim('',str(rec['meta_id']),user_rep) + meta_sim('',str(rec['meta_id']),user_rep)+artist_sim('',str(rec['artist_id']),user_rep) +raag_sim('',str(rec['raag_id']),user_rep))/4
        #score =(artist_sim('',str(rec['artist_id']),user_rep)+meta_sim('',str(rec['meta_id']),user_rep) + rec_sim('',str(rec['meta_id']),user_rep))/3
        heapq.heappush(priority_queue, (-score,rec['meta_id']))  #max-heap
    
    #Display in descending order, using audio player allow the user to play the rec.
    recs = []
    for i in range(min(10,len(priority_queue))):
        recs.append(heapq.heappop(priority_queue))
    
    
    return recs


In [574]:
rec_listraag = getRaagRec('Sohini',['313','761','12'])
#rec_listraag = getRaagRec('Sohini',['307','761','29'])

for i, rec in enumerate(rec_listraag):
    getAudioDetails(rec[1])


 Id:303, Artist : Ulhas Kashalkar , Main Instrument : Vocal , Raga : Sohini , Taal : nan , Laya : VilambitDrut
 Id:257, Artist : Sameehan Kashalkar , Main Instrument : Vocal , Raga : Sohini , Taal : nan , Laya : nan
 Id:146, Artist : Milind Malshe , Main Instrument : Vocal , Raga : Sohini , Taal : nan , Laya : nan
 Id:234, Artist : Roopa Panesar , Main Instrument : Sitar , Raga : Sohini , Taal : Addha , Laya : Madhya


In [ ]:
meta_dataset

In [ ]:
def getArtistRec(artist,user_rep):
    
    # Filter rows where the value in the "Raga" column equals the given raag
    raag_rows = meta_dataset[meta_dataset['Artist'] == artist]
    
    # Get the indexes of the filtered rows
    indexes = raag_rows.index.tolist()
    
    priority_queue = []
    
    customdf = []
    for meta_id in indexes:
        row =id_dataset.loc[meta_id].tolist()
        customdf.append(row)
    
    id_data = pd.DataFrame(customdf, columns=['meta_id','artist_id','raag_id'])
    for index,rec in id_data.iterrows():
            
        score =(rec_sim('',str(rec['meta_id']),user_rep) + meta_sim('',str(rec['meta_id']),user_rep)+artist_sim('',str(rec['artist_id']),user_rep) +raag_sim('',str(rec['raag_id']),user_rep))/4
        
        heapq.heappush(priority_queue, (-score,rec['meta_id']))  #max-heap
    
    user_history = user.get_liked_recs()
    print(user_history)
    #Display in descending order, using audio player allow the user to play the rec.
    recs = []
    for i in range(min(10,len(priority_queue))):
        recs.append(heapq.heappop(priority_queue))

    
    return recs


In [ ]:
rec_listraag = getArtistRec('Ulhas Kashalkar',['313','761','29'])
#rec_listraag = getRaagRec('Sohini',['307','761','29'])

for i, rec in enumerate(rec_listraag):
    getAudioDetails(rec[1])


In [562]:
#data
# cluster_centres = []
data_in_clusters = [[]] #2d array

#similarities
complete_similarities = [[]] #2d array

In [563]:
meta_dataset

,Raga,Duration,Artist,Instrument,Laya,Taal,Accompanists,Instruments,Form,Name
0,Sahana,474,Abhisek Lahiri,Sarod,Madhya,Jhaptaal,Sanju Sahai,Tabla,Khyal,RaagSahanaAbhisekLahiriMusicofIndia
1,Shree,713,Abhisek Lahiri,Sarod,Drut,Teentaal,Sukhvinder Singh 'Pinky',Tabla,Khyal,AbhisekLahiriSukhvinderSinghPinkyRaagShreeSaro...
2,Malkauns,247,Abhisek Lahiri,Sarod,Drut,NaN,NaN,NaN,Jor|Jhala,SpellbindingRaagMalkaunsAbhisekLahiriMusicofIndia
3,Kaunsi Kanada,59,Abhisek Lahiri,Sarod,Drut,NaN,NaN,NaN,Khyal,RaagKaunsiKanadaAbhisekLahiriMusicofIndia
4,Chayanat,598,Abhisek Lahiri,Sarod,Madhya,Roopak,Sukhvinder Singh 'Pinky',Tabla,Khyal,AbhisekLahiriSukhvinderSinghPinkyRaagChayanatS...
...,...,...,...,...,...,...,...,...,...,...
308,Multani,657,Venkatesh Kumar,Vocal,Vilambit,Ektaal,Murad Ali Khan|Satyajit Talwalkar|Tofail Ahmed,Sarangi|Tabla|Harmonium,Khyal,AmazingRaagMultaniPanditVenkateshKumarSarangiH...
309,Multani,335,Venkatesh Kumar,Vocal,Drut,Teentaal,Satyajit Talwalkar|Murad Ali|Tofail Ahmed,Tabla|Sarangi|Harmonium,Khyal,PowerfulKhayalRaagMultaniPanditVenkateshKumarM...
310,Patdeep,327,Venkatesh Kumar,Vocal,Madhya,Jhaptal,Murad Ali Khan|Satyajit Talwalkar|Tofail Ahmed,Sarangi|Tabla|Harmonium,Khyal,MajesticRaagPatdeepVenkateshKumarKhayalvocal
311,Shuddha Sarang,429,Vinayak Torvi,Vocal,Madhya,Teentaal,Kousic Sen|Bharat Bhushan Goswami|Ajay Joglekar,Tabla|Sarangi|Harmonium,Khyal,PanditVinayakTorviRaagShudhSarangKhayalVocal


In [564]:
def get_file_path(audio_name, folder_path):
    # Concatenate ".mp3" to the audio name
    audio_file = audio_name+"mp.mp3"
    # Construct the full path to the file
    file_path = os.path.join(folder_path, audio_file)

    # Check if the file exists
    if os.path.isfile(file_path):
        return file_path


In [573]:
def getAudioDetails(metaID):
    row = meta_dataset.loc[metaID]
    
    artists = str(row['Artist']).split('|')
    row_artists = ""
    for artist in artists:
        row_artists +=artist
    
    instruments = str(row['Instrument']).split('|')
    row_instruments = ""
    for instrument in instruments:
        row_instruments +=instrument
        
    ragas = str(row['Raga']).split('|')
    row_ragas = ""
    for raga in ragas:
        row_ragas +=raga
    
    taals = str(row['Taal']).split('|')
    row_taals = ""
    for taal in taals:
        row_taals += taal
    
    layas = str(row['Laya']).split('|')
    row_layas = ""
    for laya in layas:
        row_layas += laya
    
    acc_instruments = str(row['Instruments']).split('|')
    row_acc_instruments = ""
    for acc_instrument in acc_instruments:
        row_acc_instruments += acc_instrument
        
    #audio_path= meta_dataset.loc[metaID,'AudioPath']
    #audio_name = os.path.basename(audio_path)
        
    print(" Id:"+ str(metaID) +", Artist : "+row_artists+" , Main Instrument : "+row_instruments+" , Raga : "+row_ragas+" , Taal : "+row_taals+" , Laya : "+row_layas)
    

In [566]:
def getAudio(metaID):
    audio_path= meta_dataset.loc[metaID,'AudioPath']
    print(audio_path)
    audio,sr = librosa.core.load(audio_path, sr=44100, mono=True)   #need to use the full path for this
    try:
        with open(audio_path, 'rb') as f:
            audio_data = f.read()
            player(audio_data,sr)
    except OSError as e:
        print(f"Error playing audio: {e}")

# def getAudio(metaID):
#     audio_path = meta_dataset.loc[metaID, 'AudioPath']
#     playsound.playsound(audio_path)

In [578]:
getAudioDetails(30)
#audio = getAudio(30)


 Id:30, Artist : Anupama Bhagwat , Main Instrument : Sitar , Raga : Sahana , Taal : nan , Laya : Drut


In [ ]:
# artistID = input("Enter an artist of your choice: ")
ragaID = input("Enter a raag of your choice: ")


user = User("1")
user.user_rep(artistID,ragaID)

print(user.get_liked_recs())
user_liked = user.get_liked_recs()
print(user_liked[0])
recs_list = giveRecsForNewVec(user_liked,"")
print("Recommendations based on user input:")
for i, rec in enumerate(recs_list):
    getAudioDetails(rec[1])

In [ ]:
artist = 'Ajoy Chakraborty'
artistID = next(key for key, value in artist_dict.items() if value == artist)
artistID

In [568]:
artist_dict['25']

'Ajoy Chakraborty'

In [588]:
artist_dict
revartist_dict = {value: key for key, value in artist_dict.items()}

In [598]:
raag_dict = {}
for index , row in raagdataset.iterrows():
    raag_dict[row['raag']] = index

In [ ]:
#### Main function
def main():

    artist = input("Enter an artist of your choice: ")
    #artistID = next(key for key, value in artist_dict.items() if value == artist)
    #artistID ='25'
    artistID = str(revartist_dict[artist])
    raga = input("Enter a raag of your choice: ")
    ragaID = str(raag_dict[raga])
    
    
    
    user = User("1")
    user.user_rep(artistID,ragaID)
#giveRecsForNewVec(user_rep,vec,raag,artist)
    user_liked = user.get_liked_recs()
    #print(user_liked)artist,user_rep)
    recs_list = giveRecsForNewVec(['','',''],user_liked)
    print("Recommendations based on user input:")
    for i, rec in enumerate(recs_list):
        getAudioDetails(rec[1])

    # Main loop
    while True:
        print("\nOptions:")
        print("1. Select one of the recommendations")
        print("2. Search for a raag")
        print("3. Search for an artist")
        print("4. Exit")

        choice = input("Enter your choice: ")
        if choice == '1':
            index = int(input("Enter the index of the recommendation you want to select: "))
            if 0 <= index < 313:
                #vec1 = getUserVector(recs_list[index])
                vec1 = id_dataset.loc[index].tolist()
                user.like_rec(vec1)
                recs_list = giveRecsForNewVec(vec1,user.get_liked_recs())
                print("Recommendations based on user input:")
                for i, rec in enumerate(recs_list):
                    if i == 0:
                        continue
                    getAudioDetails(rec[1])
                #getTimeRec()
            else:
                print("Invalid index.")
        elif choice == '2':
            search_term = input("Enter the raag you want to search for: ")
            
            rec_listraag = getRaagRec(search_term, user.get_liked_recs())
            for i, rec in enumerate(rec_listraag):
 
                getAudioDetails(rec[1])
                #getTimeRec()
            
        elif choice == '3':
            search_term = input("Enter the artist you want to search for: ")
            
            rec_listraag = getArtistRec(search_term, user.get_liked_recs())
            for i, rec in enumerate(rec_listraag):
                getAudioDetails(rec[1])
            
        
        elif choice == '4':
            print("Exiting the program.")
            break
        else:
            print("Invalid choice. Please enter a valid option.")

if __name__ == "__main__":
    main()

Enter an artist of your choice:  Shahid Parvez
Enter a raag of your choice:  Yaman


Recommendations based on user input:
 Id:273, Artist : Shahid Parvez , Main Instrument : SitarTabla , Raga : Yaman , Taal : Teentaal , Laya : DrutMadhya
 Id:274, Artist : Shahid Parvez , Main Instrument : Sitar , Raga : Yaman , Taal : nan , Laya : Drut
 Id:272, Artist : Shahid Parvez , Main Instrument : Sitar , Raga : Piloo , Taal : nan , Laya : Vilambit
 Id:57, Artist : Budhaditya Mukherjee , Main Instrument : Sitar , Raga : Hameer , Taal : Teentaal , Laya : Madhya
 Id:287, Artist : Budhaditya Mukherjee , Main Instrument : Sitar , Raga : Nand , Taal : Teentaal , Laya : Vilambit
 Id:33, Artist : Anupama Bhagwat , Main Instrument : Sitar , Raga : Shyam Kalyan , Taal : nan , Laya : VilambitDrut
 Id:4, Artist : Abhisek Lahiri , Main Instrument : Sarod , Raga : Chayanat , Taal : Roopak , Laya : Madhya
 Id:112, Artist : Kaustuv Kanti Ganguli , Main Instrument : Vocal , Raga : Yaman , Taal : nan , Laya : nan
 Id:174, Artist : Omkar Dadarkar , Main Instrument : Vocal , Raga : Yaman , Taal : T

Enter your choice:  1
Enter the index of the recommendation you want to select:  57


Recommendations based on user input:
 Id:287, Artist : Budhaditya Mukherjee , Main Instrument : Sitar , Raga : Nand , Taal : Teentaal , Laya : Vilambit
 Id:273, Artist : Shahid Parvez , Main Instrument : SitarTabla , Raga : Yaman , Taal : Teentaal , Laya : DrutMadhya
 Id:4, Artist : Abhisek Lahiri , Main Instrument : Sarod , Raga : Chayanat , Taal : Roopak , Laya : Madhya
 Id:0, Artist : Abhisek Lahiri , Main Instrument : Sarod , Raga : Sahana , Taal : Jhaptaal , Laya : Madhya
 Id:293, Artist : Swapan Chaudhuri , Main Instrument : Sitar , Raga : Alhaiya Bilawal , Taal : Teentaal , Laya : Madhya
 Id:71, Artist : Prabha Atre , Main Instrument : Vocal , Raga : Shyam Kalyan , Taal : Ektaal , Laya : Madhya
 Id:49, Artist : Brajeshwar Mukherjee , Main Instrument : Vocal , Raga : Hameer , Taal : EktaalJhaptaal , Laya : Madhya
 Id:88, Artist : Kala Ramnath , Main Instrument : Violin , Raga : Tilak Kamod , Taal : Jhaptal , Laya : Madhya
 Id:234, Artist : Roopa Panesar , Main Instrument : Sitar 

Enter your choice:  1
Enter the index of the recommendation you want to select:  88


Recommendations based on user input:
 Id:57, Artist : Budhaditya Mukherjee , Main Instrument : Sitar , Raga : Hameer , Taal : Teentaal , Laya : Madhya
 Id:89, Artist : Kala Ramnath , Main Instrument : Violin , Raga : Puriya Dhanashree , Taal : Teentaal , Laya : Madhya
 Id:295, Artist : Tejendra Majumdar , Main Instrument : Sarod , Raga : Tilak Kamod , Taal : Teentaal , Laya : Madhya
 Id:287, Artist : Budhaditya Mukherjee , Main Instrument : Sitar , Raga : Nand , Taal : Teentaal , Laya : Vilambit
 Id:39, Artist : Ashwini Bhide-Deshpande , Main Instrument : Vocal , Raga : Jansammohini , Taal : Addha , Laya : Madhya
 Id:0, Artist : Abhisek Lahiri , Main Instrument : Sarod , Raga : Sahana , Taal : Jhaptaal , Laya : Madhya
 Id:15, Artist : Ajoy Chakraborty , Main Instrument : Vocal , Raga : Kalavati , Taal : RoopakTeentaal , Laya : Madhya
 Id:49, Artist : Brajeshwar Mukherjee , Main Instrument : Vocal , Raga : Hameer , Taal : EktaalJhaptaal , Laya : Madhya
 Id:4, Artist : Abhisek Lahiri , M

Enter your choice:  2
Enter the raag you want to search for:  Todi


 Id:20, Artist : Ajoy Chakraborty , Main Instrument : Vocal , Raga : Todi , Taal : JhoomraTeentaal , Laya : Vilambit
 Id:183, Artist : Omkar Dadarkar , Main Instrument : Vocal , Raga : Todi , Taal : TeentaalTeentaal , Laya : Madhya
 Id:80, Artist : Indrani Mukherjee , Main Instrument : Vocal , Raga : Todi , Taal : nan , Laya : Vilambit
 Id:279, Artist : Shivputra Shidhharamayya Komkali 'Kumar Gandharva' , Main Instrument : Vocal , Raga : Todi , Taal : EktaalTeentaal , Laya : nan

Options:
1. Select one of the recommendations
2. Search for a raag
3. Search for an artist
4. Exit


Enter your choice:  2
Enter the raag you want to search for:  Bhimpalasi


 Id:164, Artist : Niladri Kumar , Main Instrument : Sitar , Raga : Bhimpalasi , Taal : Dhamar , Laya : Madhya
 Id:286, Artist : Shubha Mudgal , Main Instrument : Vocal , Raga : Bhimpalasi , Taal : Teentaal , Laya : Madhya
 Id:125, Artist : Kushal Das , Main Instrument : Sitar , Raga : Bhimpalasi , Taal : Teentaal , Laya : Drut
 Id:121, Artist : Kushal Das , Main Instrument : Sitar , Raga : Bhimpalasi , Taal : Teentaal , Laya : Vilambit
 Id:124, Artist : Kushal Das , Main Instrument : Sitar , Raga : Bhimpalasi , Taal : Teentaal , Laya : Vilambit
 Id:96, Artist : Kaushiki Chakraborty Desikan , Main Instrument : Vocal , Raga : Bhimpalasi , Taal : Ektaal , Laya : Vilambit
 Id:83, Artist : Jayateerth Mevundi , Main Instrument : Vocal , Raga : Bhimpalasi , Taal : Teentaal , Laya : Madhya
 Id:99, Artist : Kaushiki Chakraborty Desikan , Main Instrument : Vocal , Raga : Bhimpalasi , Taal : Teentaal , Laya : Drut
 Id:208, Artist : Prem Kumar MallickPrashant Mallick , Main Instrument : VocalVocal

In [ ]:
similarity_matrix = np.load('path/to/your/file.npy')